In [48]:
import json
import os
import numpy as np
import pandas as pd
import emoji

root = "sejal"

def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return True
    return False

identifier = []
photos = []
captions = []
collections = 0
no_image = 0
no_caption = 0
has_quotes = 0
scraper_failed = 0
posts_per_user = []
mentions = 0
hashtags = 0
emojis = 0
vanilla = 0
for fn in os.listdir("profiles_" + root + "/"):
    if fn.endswith(".json"):
        f = open("profiles_" + root + "/" + fn)
        num_valid_entries = 0
        num_invalid_entries = 0
        data = json.load(f)
        try:
            posts = data["posts"]
        except:
            if data["num_of_posts"] >= 0 and data["num_of_posts"] <= 100:
                print(data["num_of_posts"])
                scraper_failed += data["num_of_posts"]
            else:
                scraper_failed += 100
        for p in posts:
            if len(p["imgs"]) > 1:
                collections += 1
                num_invalid_entries += 1
            elif len(p["imgs"]) == 0:
                no_image += 1
                num_valid_entries += 1
            elif p["caption"] == "":
                no_caption += 1
                num_invalid_entries += 1
            else:
                num_valid_entries += 1
                vanilla_flag = True
                identifier.append(data["username"])
                photos.append(p["imgs"][0])
                if '"' in p["caption"]:
                    has_quotes += 1
                    vanilla_flag = False
                if '@' in p["caption"]:
                    mentions += 1
                    vanilla_flag = False
                if '#' in p["caption"]:
                    hashtags += 1
                    vanilla_flag = False
                if text_has_emoji(p["caption"]):
                    emojis += 1
                    vanilla_flag = False
                if vanilla_flag:
                    vanilla += 1
                captions.append(p["caption"].replace('"', '\''))
        posts_per_user.append(num_valid_entries)
        if data["num_of_posts"] < 100:
            scraper_failed += data["num_of_posts"] - num_valid_entries - num_invalid_entries
        else:
            scraper_failed += 100 - num_valid_entries - num_invalid_entries
        f.close()

total = len(captions) + collections + no_image + no_caption
print("scraper failed", scraper_failed, round(scraper_failed * 100/ (total + scraper_failed), 2))
print("throwing out:", collections + no_image + no_caption, round((collections + no_image + no_caption) * 100 / total, 2))
print("\t no caption", no_caption, round((no_caption / total) * 100, 2))
print("\t no image", no_image, round((no_image / total) * 100, 2))
print("\t collection", collections, round((collections / total) * 100, 2))
print("usable:", len(captions), round(len(captions)*100/total, 2))
print("\t mentions", mentions, round((mentions / total) * 100, 2))
print("\t hashtags", hashtags, round((hashtags / total) * 100, 2))
print("\t emojis", emojis, round((emojis / total) * 100, 2))
print("\t has quotes", has_quotes, round((has_quotes / total) * 100, 2))
print("\t vanilla", vanilla, round((vanilla / total) * 100, 2))
print("post per user:", posts_per_user, len(posts_per_user))
print("\t average posts per user:", np.mean(posts_per_user))
print("\t median posts per user:", np.median(posts_per_user))
print("\t number of posts that would be thrown out:", sum([p if (p < 10 and p > 0) else 0 for p in posts_per_user]))

with open('data_' + root + '.csv', 'w') as f:
    f.write("username,photo,caption\n")
    for i in range(len(photos)):
        f.write("%s,%s,\"%s\"\n" % (identifier[i],photos[i], captions[i]))
f.close()

try:
    csv = pd.read_csv('data_' + root + '.csv', header=0)
    csv.drop_duplicates(keep="last",inplace=True)
    csv.to_csv('data_' + root + '.csv', index=False)
except:
    print('pandas stuff did not work')


scraper failed 21632 33.79
throwing out: 14244 33.61
	 no caption 3317 7.83
	 no image 934 2.2
	 collection 9993 23.58
usable: 28134 66.39
	 mentions 2126 5.02
	 hashtags 4739 11.18
	 emojis 10263 24.22
	 has quotes 465 1.1
	 vanilla 13162 31.06
post per user: [6, 48, 0, 13, 11, 70, 1, 41, 0, 7, 77, 54, 78, 54, 4, 94, 16, 17, 32, 1, 3, 17, 5, 9, 10, 48, 53, 0, 0, 72, 12, 11, 2, 55, 27, 59, 16, 56, 70, 44, 0, 5, 0, 10, 11, 24, 0, 1, 18, 89, 9, 3, 88, 11, 1, 4, 28, 39, 13, 24, 12, 44, 0, 6, 6, 33, 80, 1, 1, 10, 0, 12, 0, 10, 0, 6, 15, 0, 14, 48, 0, 15, 5, 39, 7, 64, 0, 10, 4, 4, 73, 1, 49, 0, 17, 75, 59, 72, 7, 8, 60, 11, 28, 17, 26, 22, 71, 12, 8, 0, 0, 10, 9, 11, 0, 0, 60, 62, 70, 0, 0, 70, 22, 63, 9, 9, 47, 57, 41, 6, 22, 0, 0, 33, 5, 8, 6, 81, 77, 9, 82, 72, 13, 0, 14, 1, 5, 81, 38, 5, 4, 5, 0, 0, 8, 64, 4, 52, 10, 13, 0, 3, 0, 19, 47, 21, 70, 71, 4, 71, 74, 3, 5, 8, 0, 20, 15, 87, 51, 5, 12, 0, 7, 1, 54, 5, 33, 82, 1, 55, 29, 0, 27, 58, 15, 9, 0, 3, 19, 6, 37, 0, 9, 47, 0, 46, 58, 7